In [37]:
# Load libraries
import os
os.environ['QT_API'] = 'pyqt4'  # or 'pyside2'
import numpy as np
import nibabel as nib
import pandas as pandas
from surfer import Brain
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


## Tau Time Series

In [150]:
# Load tau time series file 

# EBM time series
#data = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/EBM time series/EBM tau time series - no tmax (for figure 1).csv')
#data = data.drop('Unnamed: 0', axis=1)

# Single subject raw tau (figure 4: 2373, figure 7: 6284)
#data_new = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/Pysurfer data/4197 raw tau.csv')

# Single subject seed op Nexis fit (figure 4: 2373, figure 7: 6284)
# data_new = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/Pysurfer data/4197 seed op.csv')

# Average otpimized seed across subjects
#data_new = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/Pysurfer data/Average individually optimized seed - version 11.csv')

# Load top U vectors
data_new = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/Pysurfer data/Top U Vectors - version 11.csv')

# Need additional steps for any data labeled "data_new"
data_new.insert(3, 'Left-Caudate', 0)
data_new.insert(4, 'Left-Putamen', 0)
data_new.insert(5, 'Left-Pallidum', 0)
data_new.insert(8, 'Left-Accumbens-area', 0)
data_new.insert(12, 'Right-Caudate', 0)
data_new.insert(13, 'Right-Putamen', 0)
data_new.insert(14, 'Right-Pallidum', 0)
data_new.insert(17, 'Right-Accumbens-area', 0)
data_new = data_new.drop('Unnamed: 0', axis=1)
norm_to_ADNI = [52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
data_new = data_new.iloc[:, norm_to_ADNI]
data = data_new

# Reorder to match DK
ADNI_to_DK = [34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85]
data = data.iloc[:, ADNI_to_DK]

# ENTER TIME POINT HERE (row #) for tau visualization and create a data frame
tau = data.iloc[3,:]

# Prepare data to match annotation file
tau = tau[:-52]

# New values and labels to be inserted
new_value_at_0 = 0
new_label_at_0 = 'unknown'  # Assuming 'a' is the label for the new entry at index 1

new_value_at_4 = 0
new_label_at_4 = 'corpuscallosum'  # Assuming 'a' is the label for the new entry at index 4 (adjusted index due to the first insertion)

# Splitting the series into parts around the insertion points
part_between_0_and_4 = tau.iloc[0:3]
part_after_4 = tau.iloc[3:]

# Inserting the new values by creating new Series for them (with labels) and concatenating everything
tau = pandas.concat([
    pandas.Series([new_value_at_0], index=[new_label_at_0]),  # Insert new value with label at adjusted index 1
    part_between_0_and_4,
    pandas.Series([new_value_at_4], index=[new_label_at_4]),  # Insert new value with label at adjusted index 4
    part_after_4
])

## Variance

In [7]:
# Model

# Load variance (of optimized seeds)
model_variance = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/Pysurfer data/Model Seed Op Variance - version 11.csv')

# Housekeeping 
model_variance = model_variance.drop('Unnamed: 0', axis=1)

# Add back in striatal regions and set to 0
model_variance.insert(2, 'Left-Caudate', 0, allow_duplicates=False)
model_variance.insert(3, 'Left-Putamen', 0, allow_duplicates=False)
model_variance.insert(4, 'Left-Pallidum', 0, allow_duplicates=False)
model_variance.insert(7, 'Left-Accumbens-area', 0, allow_duplicates=False)
model_variance.insert(11, 'Right-Caudate', 0, allow_duplicates=False)
model_variance.insert(12, 'Right-Putamen', 0, allow_duplicates=False)
model_variance.insert(13, 'Right-Pallidum', 0, allow_duplicates=False)
model_variance.insert(16, 'Right-Accumbens-area', 0, allow_duplicates=False)

# Reorder to match DK
new_order = [18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
model_variance = model_variance.iloc[:, new_order]

# Prepare data to match annotation file
model_variance.insert(0, 'unknown', 0, allow_duplicates=False)
model_variance.insert(4, 'corpuscallosum', 0, allow_duplicates=False)
#model_variance = model_variance.iloc[:, :-52]
model_variance = model_variance.iloc[0].squeeze()


In [8]:
# EBM

# Load variance (of raw empirical tau data)
ebm_variance = pandas.read_csv('/Users/robinsandell/Desktop/Raj Lab/EBM + Nexis/CODE EBM + Nexis/Nexis/Pysurfer data/EBM Variance - version 11.csv')

# Housekeeping 
ebm_variance = ebm_variance.drop('Unnamed: 0', axis=1)

# Add back in striatal regions and set to 0
ebm_variance.insert(2, 'Left-Caudate', 0, allow_duplicates=False)
ebm_variance.insert(3, 'Left-Putamen', 0, allow_duplicates=False)
ebm_variance.insert(4, 'Left-Pallidum', 0, allow_duplicates=False)
ebm_variance.insert(7, 'Left-Accumbens-area', 0, allow_duplicates=False)
ebm_variance.insert(11, 'Right-Caudate', 0, allow_duplicates=False)
ebm_variance.insert(12, 'Right-Putamen', 0, allow_duplicates=False)
ebm_variance.insert(13, 'Right-Pallidum', 0, allow_duplicates=False)
ebm_variance.insert(16, 'Right-Accumbens-area', 0, allow_duplicates=False)

# Reorder to match DK
new_order = [18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
ebm_variance = ebm_variance.iloc[:, new_order]

# Prepare data to match annotation file
ebm_variance.insert(0, 'unknown', 0, allow_duplicates=False)
ebm_variance.insert(4, 'corpuscallosum', 0, allow_duplicates=False)
#ebm_variance = ebm_variance.iloc[:, :-52]
ebm_variance = ebm_variance.iloc[0].squeeze()

## Brain Visualization

In [127]:
# Define "SUBJECTS_DIR
import os

os.environ["SUBJECTS_DIR"] = "/Applications/freesurfer/subjects"

In [152]:
# OPTION 1: Visualize cortical surface (using uploaded ROI data)

print(__doc__)

blue_cmap = LinearSegmentedColormap.from_list("custom_blue", ["#E0FFFF", "#87CEFA", "#4682B4", "#00008B"])
orange_cmap =LinearSegmentedColormap.from_list("yellow", ["#f4cca1", "#ff8c69","#ff5533", "#ff2200", "#760909"])
combined_cmap = LinearSegmentedColormap.from_list("blue_to_orange", ["#00008B", "#4682B4", "#87CEFA", "#E0FFFF", "#f4cca1", "#ff8c69", "#ff2200", "#760909"]) 


subject_id = "fsaverage"
hemi = "lh"
surf = "pial" # Other options: "pial" produces more anatomically detailed surface, "inflated" produces smooth surface

"""
Bring up the visualization.
"""

# Initiate Brain object
brain = Brain(subject_id, hemi, surf, background="white", subjects_dir='/Applications/freesurfer/subjects')

"""
Read in the automatic parcellation of sulci and gyri.
"""

# Load annotation file
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

"""
Make a random vector of scalar data corresponding to a value for each region in
the parcellation.

"""

# ENTER DATA HERE ('model variance,' 'ebm_variance', 'tau', etc.)
roi_data = tau #'tau' or 'ebm_variance' or 'model variance'

"""
Make a vector containing the data point at each vertex.
"""
vtx_data = roi_data[labels]

"""
Handle vertices that are not defined in the annotation.
"""
vtx_data[labels == -1] = -1

"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
brain.add_data(vtx_data, -0.2, 0.2, thresh=0, colormap=combined_cmap, alpha=0.8) # EDIT alpha transparancy level here (0-1)
#vec 1: 0, 0.25
#vec 2: -0.2, 0.2
#vec 3: -0.15, 0.3
#vec 4: -0.2, 0.2

# Select desired view of brain
brain.show_view('medial')


Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.



/var/folders/bd/p6dnm7ns0tb9glfgc_h835k40000gn/T/ipykernel_50293/1027481287.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vtx_data = roi_data[labels]
Exception occurred in traits notification handler for object: <mayavi.filters.threshold.Threshold object at 0x2b42b6130>, trait: upper_threshold, old value: 1e+20, new value: 0.2089547309708296
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/traits/trait_notifiers.py", line 342, in __call__
    self.handler(*args)
  File "/opt/homebrew/lib/python3.9/site-packages/mayavi/filters/threshold.py", line 174, in _upper_threshold_changed
    fil.threshold_between(self.lower_threshold, new_value)
  File "/opt/homebrew/lib/python3.9/site-packages/tvtk/tvtk_classes.zip/tvtk_classes/threshold.py", line 257, in threshold_be

((0.0,
  90.0,
  347.5779724121094,
  array([-33.50578773, -17.93186188,  15.4875679 ])),
 -90.0)

In [20]:
# OPTION 2: Plot interior subcortical structures

print(__doc__)

subject_id = "fsaverage"
hemi = "lh"
surf = "pial"

# Initiate brain object
brain = Brain(subject_id, hemi, surf, cortex='ivory', background="white", alpha=0.8) # EDIT alpha transparancy level here (0-1)

# Manually enter the highest tau regions MNI coordinates and their intensity (in this case, their variance)
coords_and_intensities = [
    ([50, -60, 0], 1), # right middle temporal
    ([-20, -10, -28], 0.9), # left entorhinal
    ([50, -30, -5], 0.8), # right bank of sts
    ([60, -10, -10], 0.7), # right superior temporal gyrus (anterior)
    ([60, -20, 0], 0.7), # right superior temporal gyrus (middle)
    ([60, -40, 10], 0.7), # right superior temporal gyrus (posterior)
    ([20, -10, -28], 0.6), # right entorhinal
    ([50, -60, 40], 0.5), # right inferior parietal (angular gyrus)
    ([60, -40, 40], 0.5), # right inferior parietal (supramarginal gyrus)
    ([-50, -40, -20], 0.4) # left inferior temporal gyrus
]

colors = [
    (0.8, 0.0, 0.0),    # Dark Red
    (1.0, 0.0, 0.0),    # Red
    (1.0, 0.4, 0.4),    # Light Red
    (1.0, 0.647, 0.0),  # Orange
    (1.0, 0.647, 0.0),  # Orange
    (1.0, 0.647, 0.0),  # Orange
    (1.0, 0.804, 0.4),  # Light Orange
    (1.0, 1.0, 0.0),    # Yellow
    (1.0, 1.0, 0.0),    # Yellow
    (1.0, 1.0, 0.4),    # Light Yellow
]

# Plot each focus point individually with specified colors
# for i, (coord, _) in enumerate(coords_and_intensities):
#     color = colors[i]
#     brain.add_foci([coord], color=color)

# Show the brain with the plotted foci
#brain.show_view('lateral')


Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.

